**This file contains functions for predicting precipitation images for 24 hour duration using the SINDy.**

**Input: 10 radar frames**

**Output: The CSI values for prediction for 1 hr**

For updating paper as per reviewer comments average CSI for a number of predictions 
for a 24 hr duration is considered. Therefore, only the CSI obtained for each prediction is saved. 

The output prediction images are only saved for a selected time interval due to storage space constraints.
    
Authors: Divya S. Vidyadharan, Aaron Xavier

Code created on 05.02.21

Last updated on 09.03.21

In [337]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import time
import cv2

from pySINDy.sindypde import SINDyPDE
from pySINDy import SINDy
from pySINDy.sindybase import SINDyBase
from pySINDy.sindylibr import SINDyLibr
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import csv
import wradlib.ipol as ipol # for interpolation
from matplotlib.cm import get_cmap
os.environ['PROJ_LIB'] = '/anaconda3/pkgs/proj4-5.2.0-h0a44026_1/share/proj/' 
# from mpl_toolkits.basemap import Basemap
import imageio
import time

In [338]:
# !pip install semver
# !pip install wradlib
module_path = os.path.abspath(os.path.join('/home/divya/rainymotion/'))
if module_path not in sys.path:
    sys.path.append(module_path)

    
# module_path = os.path.abspath(os.path.join('..'))
# if module_path not in sys.path:
#     sys.path.append(module_path)
from rainymotion import metrics
from rainymotion.metrics import *
# 

**Advection Schemes**

In [339]:

def _advection_constant_vector(u,v, lead_steps=12):

    delta_x = u #of_instance[::, ::, 0]
    delta_y = v #of_instance[::, ::, 1]

    # make a source meshgrid
    coord_source_i, coord_source_j = np.meshgrid(range(u.shape[1]),
                                                 range(u.shape[0]))

    # calculate new coordinates of radar pixels
    coord_targets = []
    for lead_step in range(lead_steps):
        coord_target_i = coord_source_i + delta_x * (lead_step + 1)
        coord_target_j = coord_source_j + delta_y * (lead_step + 1)
        coord_targets.append([coord_target_i, coord_target_j])

    coord_source = [coord_source_i, coord_source_j]

    return coord_source, coord_targets


# semi-Lagrangian advection
def _advection_semi_lagrangian(u,v, lead_steps=12):

    delta_x = u #of_instance[::, ::, 0]
    delta_y = v # of_instance[::, ::, 1]

    # make a source meshgrid
    coord_source_i, coord_source_j = np.meshgrid(range(u.shape[1]),
                                                 range(u.shape[0]))

    # create dynamic delta holders
    delta_xi = delta_x.copy()
    delta_yi = delta_y.copy()

    # Block for calculation displacement
    # init placeholders
    coord_targets = []
    for lead_step in range(lead_steps):

        # calculate corresponding targets
        coord_target_i = coord_source_i + delta_xi
        coord_target_j = coord_source_j + delta_yi

        coord_targets.append([coord_target_i, coord_target_j])

        # now update source coordinates
        coord_source_i = coord_target_i
        coord_source_j = coord_target_j
        coord_source = [coord_source_j.ravel(), coord_source_i.ravel()]

        # update deltas
        delta_xi = map_coordinates(delta_x, coord_source).reshape(u.shape[0], u.shape[1])
        delta_yi = map_coordinates(delta_y, coord_source).reshape(u.shape[0], u.shape[1])

    # reinitialization of coordinates source
    coord_source_i, coord_source_j = np.meshgrid(range(u.shape[1]),
                                                 range(u.shape[0]))
    coord_source = [coord_source_i, coord_source_j]

    return coord_source, coord_targets


# # interpolation routine
def _interpolator(points, coord_source, coord_target, method="idw"):

    coord_source_i, coord_source_j = coord_source
    coord_target_i, coord_target_j = coord_target

    # reshape
    trg = np.vstack((coord_source_i.ravel(), coord_source_j.ravel())).T
    src = np.vstack((coord_target_i.ravel(), coord_target_j.ravel())).T

    if method == "nearest":
        interpolator = NearestNDInterpolator(src, points.ravel(),
                                             tree_options={"balanced_tree": False})
        points_interpolated = interpolator(trg)
    elif method == "linear":
        interpolator = LinearNDInterpolator(src, points.ravel(), fill_value=0)
        points_interpolated = interpolator(trg)
    elif method == "idw":
        interpolator = ipol.Idw(src, trg)
        points_interpolated = interpolator(points.ravel())

    # reshape output
    points_interpolated = points_interpolated.reshape(points.shape)

    return points_interpolated.astype(points.dtype)



In [340]:
 times=['0000','0010', '0020', '0030', '0040', '0050',
          '0100', '0110', '0120', '0130', '0140', '0150',
          '0200', '0210', '0220', '0230', '0240', '0250',
          '0300', '0310', '0320', '0330', '0340', '0350',
          '0400', '0410', '0420', '0430', '0440' ,'0450',
          '0500', '0510', '0520', '0530', '0540', '0550',
          
          '0600', '0610', '0620', '0630', '0640', '0650',
          '0700', '0710', '0720', '0730', '0740', '0750',
          '0800', '0810', '0820', '0830', '0840', '0850',
          '0900', '0910', '0920', '0930', '0940', '0950',
          '1000', '1010', '1020', '1030', '1040', '1050',
          '1100', '1110', '1120', '1130', '1140', '1150',
          '1200', '1210', '1220', '1230', '1240', '1250',
          
          
          '1300', '1310', '1320', '1330', '1340', '1350',
          '1400', '1410', '1420', '1430', '1440', '1450',
          '1500', '1510', '1520', '1530', '1540', '1550',
          '1600', '1610', '1620', '1630', '1640', '1650',
          '1700', '1710', '1720', '1730', '1740', '1750',
          '1800', '1810', '1820', '1830', '1840', '1850',
          
          '1900', '1910', '1920', '1930', '1940', '1950',
          '2000', '2010', '2020', '2030', '2040', '2050',
          '2100', '2110', '2120', '2130', '2140', '2150',
          '2200', '2210', '2220', '2230', '2240', '2250',
          '2300', '2310', '2320', '2330', '2340', '2350']
#Latitude and Longitude of Typhoon Faxai



**Our own utitilty functions for reading radar images, TVL1-SINDy u,v vectors.**

In [341]:
#For plotting map - currently using function as in source code Need to change to Cartopy
def plotMap(title,img, lat1, lat2, long1, long2, outputPath,last=0):
   
    (height, width) = img.shape

#     print(img.min(), img.max())
    intensity = np.asarray(img, dtype=np.float32)
#     print(intensity.min(), intensity.max())

    #intensity_level = convert_rep_to_level(intensity).reshape(height, width)
#     print(intensity.min(), intensity.max())
    intensity_level = np.flipud(intensity)

    dLon = (long2 - long1) / width
    dLat = (lat2 - lat1) / height

    lon = np.arange(long1, long2, dLon)
    lat = np.arange(lat1, lat2, dLat)
    lons, lats = np.meshgrid(lon, lat)
#     print(lons.shape, lats.shape)

    fig = plt.figure(figsize=(12, 8))

    # Set up Basemap instance
    m = Basemap(projection="cyl",
                llcrnrlon=long1, urcrnrlon=long2,
                llcrnrlat=lat1, urcrnrlat=lat2,
                resolution='h')

    # Add geographic outlines
    m.drawcoastlines(color='black')
    m.drawstates()
    m.drawcountries()
    m.drawmeridians(np.arange(long1, long2, 1), labels=[True, False, False, True])
    m.drawparallels(np.arange(lat1, lat2, 1), labels=[True, False, True, False])
    #m.drawmeridians(np.arange(new_lon_min, new_lon_max, 1), labels=[False, False, False, False])
    #m.drawparallels(np.arange(new_lat_min, new_lat_max, 1), labels=[False, False, False, False])


    # Plot Data
    #cs = m.contourf(lons, lats, intensity_level, shading='flat', levels=list(range(1, 65)), cmap=get_cmap("jet"))
    
    #cs = m.contourf(lons, lats, intensity_level,shading='flat', levels=list(range(1,65)), cmap=get_cmap("gist_earth"))
    cs = m.contourf(lons, lats, intensity_level,shading='flat', levels=list(range(1,65)), cmap=discrete_cmap(8,"jet"))
    # Add Colorbar
    if last==1:
        cb = plt.colorbar(cs ,shrink=1.0) #, extend='both')

    # Add Title
    plt.title(title)
    plt.savefig(outputPath, bbox_inches='tight', pad_inches=0.0)
    plt.close()



In [342]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

Following code taken from SINDy PDE written by Aaron Xavier.

In [343]:
# import pixiedust

In [344]:
# # %%pixie_debugger -b loadUV
# def loadUV(loadpath,startframetime,height,width):
#     import pdb; pdb.set_trace()
#     start_load = time.time()
#     U_mat=np.zeros((height,width))
#     V_mat=np.zeros((height,width))
#     print("\n--- Loading UV-Data.. ---\n")
#     files = (os.listdir(loadpath))
#     files.sort()
#     framecount = 0
    
#     position = 0
#     count = 0
#     data = np.zeros((2*height,width), dtype='float32')
#     u = np.zeros((height,width), dtype='float32')
#     v = np.zeros((height,width), dtype='float32')
  
#     for filename in files:
#         print("Filename here :",filename) 
#         l = filename.split('_')
#         startframe = l[1];
#         nextframe = l[3].split('.')[0]
#         print("loaduv input startframe:  ")
#         print(startframe)
#         if (l[1]>= startframetime) and framecount <20 : #lastpredtime): %10 u files and 10 v files
            
#             framecount = framecount +1
#             inputFileName =loadpath+'/'+filename
#             fd = open(inputFileName,'rb')
#             print("loaduv input:  l[1]>= startframetime ")
#             print(inputFileName)
#             # move to position in file
#             fd.seek(position,0)
#             # straight to numpy data (no buffering) 
#             data = 0.0
#             u = 0.0
#             v = 0.0
#             data = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
#             data = np.reshape(data,(2*height,width))
#             data = data.astype('float32') 
#             #print(np.dtype(data))
#             print(data.shape)
#             u = data[0:height,0:width]
#             v = data[height:,0:width]
#             fd.close()
#             U_mat = np.dstack([U_mat,u])
#             V_mat = np.dstack([V_mat,v])
        
           
#     print("\n--- Load complete.. Time: %s seconds ---\n" %(time.time() - start_load))
    
#     return U_mat, V_mat

# ##Input Path##
# pred_date = 20190908 #YYYYMMDD
# # pred_time = '0220' #HHMM 
# [height, width] = [781,561]
# inputfolder = str(pred_date)+"_set_24hr_uv"
# inputpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/%s"%inputfolder
# print (inputpath)
# startframetime = "002500"
# pred_time = '0250'
# index = times.index(pred_time)
# print(times[index-10]:index)
# # loadUV(inputpath,startframetime,height,width)

In [345]:
# file_dim = [height,width]

In [346]:
def loadUV(loadpath,pred_time,height,width):
    
    
    index = times.index(pred_time)
    prev_frametimes = times[index-10:index]
    print(prev_frametimes)
    
    
    
#     start_load = time.time()
    U_mat=np.zeros((height,width))
    V_mat=np.zeros((height,width))
    print("\n--- Loading UV-Data.. ---\n")
    files = (os.listdir(loadpath))
    files.sort()
    framecount = 0
    
    position = 0
    count = 0
    data = np.zeros((2*height,width), dtype='float32')
    u = np.zeros((height,width), dtype='float32')
    v = np.zeros((height,width), dtype='float32')
  
    for previndex in range(len(prev_frametimes)):
        searchpattern = "_" + prev_frametimes[previndex] +"00_"
        for filename in files:
            if searchpattern in filename:
                inputFileName =loadpath+'/'+filename
#                 print(searchpattern)
                fd = open(inputFileName,'rb')
                fd.seek(position,0)
                # straight to numpy data (no buffering) 
                data = 0.0
                u = 0.0
                v = 0.0
                data = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                data = np.reshape(data,(2*height,width))
                data = data.astype('float32') 
                #print(np.dtype(data))
                print(data.shape)
                u = data[0:height,0:width]
                v = data[height:,0:width]
                fd.close()
                U_mat = np.dstack([U_mat,u])
                V_mat = np.dstack([V_mat,v])
#     print("\n--- Load complete.. Time: %s seconds ---\n" %(time.time() - start_load))
    
    return U_mat, V_mat


In [347]:
def trainSINDy(U,V,dx,dy,dt):
    
    
    model = SINDyPDE(name='SINDyPDE model for Reaction-Diffusion Eqn')
    
#     start_train=time.time()
    #model.fit(self, data, poly_degree=2, cut_off=1e-3)
    model.fit({'u': U[:,:,-1*train_size:], 'v': V[:,:,-1*train_size:]}, dt, [dx, dy], space_deriv_order=2, poly_degree=2, sample_rate=0.01, cut_off=0.01, deriv_acc=2)

#     print("\n--- Train time %s seconds ---\n" %(time.time() - start_train))

    #print("\n--- Active terms ---\n" )
    size=np.shape(model.coefficients)
    cnt=0
    for i in range(size[0]):
        for j in range(size[1]):
            if (model.coefficients[i,j])!=0:
                #print(model.coefficients[i,j],"--",model.descriptions[i])
                cnt+=1
    print("Train Success..")
    print("--- Active terms %s ---\n" %cnt)
    return U, V, model.coefficients

def testSINDy(U,V,dx,dy,dt,coeff,x):

    
    model2 = SINDyLibr(name='Derived module from sindybase.py for libr computation')
    libx=model2.libr({'u': U[:,:,-1*train_size:], 'v': V[:,:,-1*train_size:]}, dt, [dx,dy], space_deriv_order=2, poly_degree=2, sample_rate=0.01, cut_off=0.01, deriv_acc=2)

    duv_dt=np.matmul(libx,coeff)

    du_dt=duv_dt[:,0]
    dv_dt=duv_dt[:,1]

    U_nxt=np.reshape(U,file_dim)+np.reshape(du_dt,file_dim)
    V_nxt=np.reshape(V,file_dim)+np.reshape(dv_dt,file_dim)
    
    Uname="%s/%i_%s_U%i.csv"%(savepath,pred_date,pred_times[x],x)
    Vname="%s/%i_%s_V%i.csv"%(savepath,pred_date,pred_times[x],x)
    np.savetxt(Uname,np.reshape(U,file_dim),delimiter=',')
    np.savetxt(Vname,np.reshape(V,file_dim),delimiter=',')
    print("Test Success..")
    #plotUV(U_nxt,V_nxt,du_dt,dv_dt,x)

    return U_nxt, V_nxt


# def gettimes24hr(pred_time):
    
   
#    # times=np.array(times)
#     inptimes = []
#     pred_times = []
#     hr = 10*int(pred_time[0]) + int(pred_time[1])
#     mi = 10*int(pred_time[2]) + int(pred_time[3])


#     for i in range(hrlimit):

#         if pred_time==times[i]:
#             for leadtime in range(leadsteps): 
#                 if (hr*(leadtime+leadsteps) < hrlimit):
#                     pos = i+leadtime+1 #hr*6+leadtime+1
#                     pred_times=np.hstack([pred_times,times[pos]])
#             for prevframes in range(totinputframes):
#                 if (i-prevframes >=0):
#                     #print(i-prevframes)
#                     inptimes=np.hstack([inptimes,times[i-prevframes]])                    
            
#                 #print("InpTimes:",inptimes[-1*(train_size+1):])
#     inptimes=inptimes[-1*(train_size+1):]
#     print("PredTimes:",pred_times)
#     print("InpTimes:",inptimes)
#     print("Get Time Success..")
    
#     return inptimes, pred_times


**To advect SINDy u,v vectors with recent radar image**

In [348]:
# Advect it using the advection schemes.
# Constant vector advection
def computeAdvection(advection,u,v,leadSteps,recentFrame,interpolation):
    # advect pixels accordingly
    if advection == "constant-vector":
#         print("Advection using constant-vector...")
        coord_source, coord_targets = _advection_constant_vector(u,v,leadSteps)
    elif advection == "semi-lagrangian":
#         print("Advection using semi-Lagrangian...")
        coord_source, coord_targets =  _advection_semi_lagrangian(u,v,leadSteps)
    
    #print("Length of coord_Source",len(coord_source)) #2
    #print(len(coord_targets))                         #18
    # nowcasts placeholder
    nowcasts = []
    # interpolation
    if leadSteps==1:
        nowcasts =_interpolator(recentFrame, coord_source,
                                      coord_targets[leadSteps-1],
                                      method=interpolation)
       
    else:
        for lead_step in range(leadSteps):
            nowcasts.append(_interpolator(recentFrame, coord_source,
                                      coord_targets[lead_step],
                                      method=interpolation))

    # reshaping
        nowcasts = np.moveaxis(np.dstack(nowcasts), -1, 0)
    #print("Shape nowcasts",nowcasts.shape)
    return nowcasts

**Get recent precipitation image for advection**

In [349]:
def getRecentRadarData(recentFramePath,recent_time,height,width,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    for fileName in files:
        if recent_time in fileName:
            if fileName.endswith(fileType):

                inputFileName = recentFramePath+'/'+fileName
#                 print(inputFileName)
                fd = open(inputFileName,'rb')
                #print(inputFileName)

                # straight to numpy data (no buffering) 

                recentFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                recentFrame = np.reshape(recentFrame,(height,width))
                recentFrame = recentFrame.astype('float16') 
    return recentFrame

**Read SINDy u,v vector files.**

In [350]:
def readSINDyFrames(inputfolderpath,height,width):
    # Read a SINDy generated u,v vector. 
    # Arrange it as a single matrix as op_instance
    
    print("Reading u,v from ",inputfolderpath)
    u = np.zeros((height,width), dtype='float32')
    v = np.zeros((height,width), dtype='float32')
    ulist = []
    vlist = []
    files = (os.listdir(inputfolderpath))
    files.sort()
    for fileName in files:
#         print(fileName)
        if fileName.endswith('.csv'):
            
            if 'U' in fileName:
                print(fileName)
                filePath = inputfolderpath+'/'+fileName
                df=pd.read_csv(filePath, sep=',',header=None)
                u[0:height,:] = df.to_numpy()
#                 print(u[0:5])
                ulist.append(u)

            if 'V' in fileName:
                filePath = inputfolderpath+'/'+fileName
                df=pd.read_csv(filePath, sep=',',header=None)
                v[0:height,:] = df.to_numpy()
                vlist.append(v)
               
    ulist = np.moveaxis(np.dstack(ulist), -1, 0)
    vlist = np.moveaxis(np.dstack(vlist), -1, 0)
#     print(ulist.shape)
    print('Read u,v frames from SINDy')
    return ulist,vlist

**To get ground truth images for performance evaluation**

In [351]:
import numpy as np
def getGroundTruthImages(pred_times,leadsteps,recentFramePath,height,width,fileType):
    files = (os.listdir(recentFramePath))
    files.sort()
    groundTruthImages = []
    
    i = 0
    while (i<leadsteps):
        
        groundtruthtime = "_"+pred_times[i]
        i = i +1
        for fileName in files:

            if groundtruthtime in fileName:
                print("The ground truth at %s is available",groundtruthtime)
                print(fileName)
                if fileName.endswith(fileType):

                    
                    inputFileName =recentFramePath+'/'+fileName
                    fd = open(inputFileName,'rb')
                    #print(inputFileName)

                    # straight to numpy data (no buffering) 
                    recentFrame = np.fromfile(fd, dtype = np.dtype('float32'), count = 2*height*width)
                    recentFrame = np.reshape(recentFrame,(height,width))
                    recentFrame = recentFrame.astype('float16') 

                    #print(recentFrame.shape)
                    groundTruthImages.append(recentFrame)
                    #else:
                    #    print("Sorry, unable to find file.")
            
    groundTruthImages = np.moveaxis(np.dstack(groundTruthImages), -1, 0)
    #print(groundTruthImages.shape)
    return groundTruthImages

**Initialization**

In [352]:
# eventName = "TropicalStormNari"
# eventDate ="20190726"
# pred_date = 20190726 #YYYYMMDD
# [height, width] = [1081, 881]

# lat1 = 30
# lat2 = 39
# long1 = 133
# long2 = 144
eventName = "TyphoonFaxai"
eventDate ="20190908"
pred_date = 20190908 #YYYYMMDD
[height, width] = [781, 561]
lat1 = 32.5
lat2 = 39
long1 = 136
long2 = 143



eventNameDate = eventName + "_" + eventDate



pred_date = 20190908 #YYYYMMDD
# pred_time = '0220' #HHMM 
[height, width] = [781,561]
file_dim = [height, width]
train_size =10

leadsteps = 6 #for 1 hr with 10 minute interval
hrlimit = len(times)#24 # 24 hr
totinputframes = 10

#Spatio-temporal resolution
x_step = 1
y_step = 1
t_step = 10.0

##Input Path##
inputfolder = str(pred_date)+"_set_24hr_uv"
inputpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/%s"%inputfolder
print (inputpath)

##recentFramePath##
recentFrameFolder = str(pred_date)+"_set_24Hr_bin" #20190908_set_24Hr_bin
recentFramePath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/%s"%recentFrameFolder
print (recentFramePath)

##Output path where predicted images for visual comparison are saved.##
outputimgpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,"SINDy","pred_images")
os.makedirs(outputimgpath, exist_ok=True)
print (outputimgpath)
loadpath = outputimgpath
##Output path where evaluation results are saved as csv files.##
outputevalpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,"SINDy","eval_results")
os.makedirs(outputevalpath, exist_ok=True)
print (outputevalpath)

savepath = outputimgpath#"Outputs/%i/%s"%(pred_date,pred_times[0])
hrlimit = len(times)
leadsteps = 6

excelpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/" + eventDate
excelpath =  excelpath + "/SINDy/eval_results/"
print("Eval results will be stored as excel workbook in ", excelpath)

/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/20190908_set_24hr_uv
/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/20190908_set_24Hr_bin
/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/SINDy/pred_images
/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/SINDy/eval_results
Eval results will be stored as excel workbook in  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/SINDy/eval_results/


In [353]:
def gettimes24hr(pred_time):
      
   # times=np.array(times)
    inptimes = []
    pred_times = []
    index = times.index(pred_time)
    indexlimit = len(times)
    print("Leadsteps are ", leadsteps)
    if (index+leadsteps) < indexlimit:
        pred_times = times[index:index+leadsteps]
    if (index-totinputframes)>=0:
        inptimes = times[index-totinputframes:index]
              
    print("PredTimes:",pred_times)
    print("InpTimes:",inptimes)
#     print("Get Time Success..")
    
    return inptimes, pred_times

In [354]:
def generateSINDy(pred_time):
#     start_prog=time.time() #tic
    

    files = (os.listdir(inputpath))
    files.sort()
    
    U,V = loadUV(inputpath,pred_time,height,width)
    #Spatio-temporal resolution pred_time
    x_step = 1
    y_step = 1
    t_step = 10.0
    train_size = 3
    totalframes = 6;
    
    for i in range(totalframes):#(len(pred_times)):
        print("\n--- Sequence %i ---\n" %i)
        #if(i==0):
        U, V, coeff = trainSINDy(U,V,x_step,y_step,t_step)
        U_nxt, V_nxt = testSINDy(U[:,:,-1:],V[:,:,-1:],x_step,y_step,t_step, coeff, i)
        U=np.dstack([U,U_nxt])
        V=np.dstack([V,V_nxt])        

    x_step = 1
    y_step = 1
    t_step = 10.0

    for i in range(len(pred_times)):
        print("\n--- Sequence %i ---\n" %i)
        #if(i==0):
        U, V, coeff = trainSINDy(U,V,x_step,y_step,t_step)
        U_nxt, V_nxt = testSINDy(U[:,:,-1:],V[:,:,-1:],x_step,y_step,t_step, coeff, i)
        U=np.dstack([U,U_nxt])
        V=np.dstack([V,V_nxt])

#     print("\n--- Exec time %s seconds ---\n" %(time.time() - start_prog)) #tac
    print("SINDy complete. Files saved to %s" %(savepath))



**SINDy u,v Interleaving with Constant Vector Advection**

In [355]:
def advectSINDyUV(saveimages):
    inputSINDyPath = outputimgpath
    modelName = "SINDyCVADV" # SINDY interleaved u,v with Constant Vector Advection
    startHr = 3
    startMin= 0
    predStartHr = 3
    step = 10
    outputpngpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/%i/%s/%s"%(pred_date,"SINDy","24HrOutputImages")
    os.makedirs(outputpngpath, exist_ok=True)

    #outputFilePath = outputFolder+'/'+modelName+'/'
    outputFilePath = outputpngpath# outputFilePath + eventNameDate
    print (outputFilePath)
    interpolation = "idw"
    advectionType = "constant-vector"
    leadSteps = 6

    ulist,vlist = readSINDyFrames(inputSINDyPath,height,width)

#     ##recentFramePath##
#     recentFrameFolder = str(pred_date)+"_set_24Hr_bin" #20190908_set_24Hr_bin
#     recentFramePath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/%s"%recentFrameFolder
#     print (recentFramePath)
    fileType = '.bin'
    for i in range(hrlimit):
        if pred_time==times[i]:
            recent_time=times[i-1]
    recent_time = "_"+recent_time #'_' to identify the recent time uniquely
    recentFrame = getRecentRadarData(recentFramePath,recent_time, height,width,fileType)
    interpolation = "idw"
    nowcastsSINDyInterCVADV = []

    for i in range(leadSteps):
        nowcastTmp = computeAdvection(advectionType,ulist[i],vlist[i],1,recentFrame,interpolation)
        nowcastsSINDyInterCVADV.append(nowcastTmp)
        recentFrame = nowcastTmp
        if saveimages:
            outFrameName = outputFilePath + '_'+str(predStartHr+(i*10))+'.png'
            matplotlib.image.imsave(outFrameName, nowcastTmp)
    nowcastsSINDyInterCVADV = np.moveaxis(np.dstack(nowcastsSINDyInterCVADV), -1, 0)
    return nowcastsSINDyInterCVADV
    

**For plotting**

In [356]:
# outNames=["2100","2110","2120","2130","2140","2150",
#               "2200","2210","2220","2230","2240", "2250",
#               "2300","2310","2320","2330","2340","2350"]
# print("Saving maps, please wait.")
# for i in range(leadSteps):    
#     outFrameName = outputFilePath + '_'+str(predStartHr+(i*10))+'.png'
#     #matplotlib.image.imsave(outFrameName, nowcastsSINDyInterCVADV[j])
#     if (i == 17) or (i == 5):
#         last = 1
#     else:
#         last = 0
#     plotMap("TVL1-SINDy Interleaved CV"+' '+str(predStartHr+(i*10)), nowcastsSINDyInterCVADV[i], lat1, lat2, long1, long2, outFrameName,last)
# print("Finished SINDy Interleaved u,v with Constant Vector Advection model nowcasting!")


**Performance Evaluation**

In [357]:
event_name = "Typhoon Faxai 09 August, 2019"
start = "21:00" 
end = "21:50"
t = ['21:00','21:10','21:20','21:30','21:40', '21:50']

In [358]:
def evaluate(nowcastsSINDy):
#     groundTruthPath = "./ForExperiments/Exp1/GroundTruth/TyphoonFaxai"
# #     groundTruthTime = "2100"
    fileType = '.bin'
#     leadsteps = 6 # 6 for 1 hr prediction, 18 for 3hr prediction
    groundTruthPath = recentFramePath
    print(pred_times)
    groundTruthImgs = getGroundTruthImages(pred_times,leadsteps,groundTruthPath,height,width,fileType)
  
    maeSINDy = []
    farSINDy = []
    podSINDy = []
    csiSINDy = []
    
    etsSINDy = []
    hssSINDy = []
    bssSINDy = []
    accSINDy = []
    precisionSINDy = []
    recallSINDy = []
    fscSINDy = []
    mccSINDy = []
    
    
    thres =1.0
    
    noOfPrecipitationImages = 6

    for i in range(noOfPrecipitationImages):
        mae = MAE(groundTruthImgs[i],nowcastsSINDy[i])
        far = FAR(groundTruthImgs[i],nowcastsSINDy[i], threshold=0.1)
        pod = POD(groundTruthImgs[i],nowcastsSINDy[i], threshold=0.1)
        csi = CSI(groundTruthImgs[i],nowcastsSINDy[i],thres)
        
        ets = ETS(groundTruthImgs[i],nowcastsSINDy[i])
        hss = HSS(groundTruthImgs[i],nowcastsSINDy[i])
        bss = BSS(groundTruthImgs[i],nowcastsSINDy[i])
        acc = ACC(groundTruthImgs[i],nowcastsSINDy[i])
        prec = precision(groundTruthImgs[i],nowcastsSINDy[i])
        rec = recall(groundTruthImgs[i],nowcastsSINDy[i])
        fsc = FSC(groundTruthImgs[i],nowcastsSINDy[i])
        mcc = MCC(groundTruthImgs[i],nowcastsSINDy[i])
        
        
        maeSINDy.append(mae)
        farSINDy.append(far)
        podSINDy.append(pod)
        csiSINDy.append(csi)
        
        etsSINDy.append(ets)
        hssSINDy.append(hss)
        bssSINDy.append(bss)
        accSINDy.append(acc)
        precisionSINDy.append(prec)
        recallSINDy.append(rec)
        fscSINDy.append(fsc)
        mccSINDy.append(mcc)
        
    return csiSINDy,maeSINDy,farSINDy,podSINDy,etsSINDy,hssSINDy, \
           bssSINDy, accSINDy, precisionSINDy, recallSINDy, fscSINDy, \
           mccSINDy


    
    

**24 Hr SINDy**

In [359]:
# pred_time = '0250'
# index = times.index(pred_time)

# prev_frametimes = times[index-10:index]
# print(prev_frametimes)

In [367]:
# %%pixie_debugger
import timeit
pred_time = '0220'#'0250'
index = times.index(pred_time)
predtimelist = []
csilist = []
maelist = []
podlist = []
farlist = []

etslist = []
hsslist = []
bsslist = []
acclist = []

precisionlist = []
recalllist = []
fsclist = []
mcclist = []

indexlimit = times.index('2250') # Since we have only 6 more ground truths available from this time
print(index)
print("Last prediction is at index ", indexlimit)
templimit = 34 # Since SINDy takes time, predictio is done for a few intervals

# index = 66 #Remove 
# pred_time ='1100'
# index = times.index(pred_time)
# print(index)
# templimit = 67 #Remove
#Added for debugging 
# intimes, pred_times = gettimes24hr(pred_time)
# generateSINDy(pred_time)
# saveimages = 0
# nowcasts = advectSINDyUV(saveimages)
# csi,mae,far,pod,ets,hss,bss, acc, prec, rec, fsc,mcc = evaluate(nowcasts)
start = timeit.timeit()
# To predict next 20
# index = 34
# pred_time = times[index]
# print(pred_time)
# templimit= 54
# index = 54
index = 70 # In the previous run could work upto 69 only
pred_time = times[index]
print(pred_time)
templimit= indexlimit
while index< templimit:#indexlimit:#templimit:#18:#len(times):
    print("\n\n Predicting for ",times[index])
    
    intimes, pred_times = gettimes24hr(pred_time)

    saveimages = 0
    if (index==66):
        saveimages=1
    generateSINDy(pred_time)
    nowcasts = advectSINDyUV(saveimages)
    csi,mae,far,pod,ets,hss,bss, acc, prec, rec, fsc,mcc = evaluate(nowcasts)
    
    predtimelist.append(pred_time)
    csilist.append(csi)
    maelist.append(mae)
    podlist.append(pod)
    farlist.append(far)
    
    etslist.append(ets)
    hsslist.append(hss)
    bsslist.append(bss)
    acclist.append(acc)
    
    precisionlist.append(prec)
    recalllist.append(rec)
    fsclist.append(fsc)
    mcclist.append(mcc)
    
    
    index = index+1
    pred_time = times[index]
#     time = times[index] 
end = timeit.timeit()
print("SINDy took ",end - start)        
print("Successfully completed predictions for ", templimit)

14
Last prediction is at index  137
1140


 Predicting for  1140
Leadsteps are  6
PredTimes: ['1140', '1150', '1200', '1210', '1220', '1230']
InpTimes: ['1000', '1010', '1020', '1030', '1040', '1050', '1100', '1110', '1120', '1130']
['1000', '1010', '1020', '1030', '1040', '1050', '1100', '1110', '1120', '1130']

--- Loading UV-Data.. ---

(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)

--- Sequence 0 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 10 ---

Dict: (438141, 66)
Test Success..

--- Sequence 1 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 83 ---

Dict: (438141, 66)
Test Success..

--- Sequence 2 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 28 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 44 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 



 Predicting for  1210
Leadsteps are  6
PredTimes: ['1210', '1220', '1230', '1240', '1250', '1300']
InpTimes: ['1030', '1040', '1050', '1100', '1110', '1120', '1130', '1140', '1150', '1200']
['1030', '1040', '1050', '1100', '1110', '1120', '1130', '1140', '1150', '1200']

--- Loading UV-Data.. ---

(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)

--- Sequence 0 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 9 ---

Dict: (438141, 66)
Test Success..

--- Sequence 1 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 81 ---

Dict: (438141, 66)
Test Success..

--- Sequence 2 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 58 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 5 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 54 ---

Dict: (438141, 66)
Test Success..



20190908_1150_U0.csv
20190908_1150_U1.csv
20190908_1200_U0.csv
20190908_1200_U1.csv
20190908_1200_U2.csv
20190908_1210_U0.csv
20190908_1210_U1.csv
20190908_1210_U2.csv
20190908_1210_U3.csv
20190908_1220_U0.csv
20190908_1220_U1.csv
20190908_1220_U2.csv
20190908_1220_U3.csv
20190908_1220_U4.csv
20190908_1230_U0.csv
20190908_1230_U1.csv
20190908_1230_U2.csv
20190908_1230_U3.csv
20190908_1230_U4.csv
20190908_1230_U5.csv
20190908_1240_U1.csv
20190908_1240_U2.csv
20190908_1240_U3.csv
20190908_1240_U4.csv
20190908_1240_U5.csv
20190908_1250_U2.csv
20190908_1250_U3.csv
20190908_1250_U4.csv
20190908_1250_U5.csv
20190908_1300_U3.csv
20190908_1300_U4.csv
20190908_1300_U5.csv
20190908_1310_U4.csv
20190908_1310_U5.csv
20190908_1320_U5.csv
Read u,v frames from SINDy
['1230', '1240', '1250', '1300', '1310', '1320']
The ground truth at %s is available _1230
20190908_123000.000.bin
The ground truth at %s is available _1240
20190908_124000.000.bin
The ground truth at %s is available _1250
20190908_125000



 Predicting for  1300
Leadsteps are  6
PredTimes: ['1300', '1310', '1320', '1330', '1340', '1350']
InpTimes: ['1120', '1130', '1140', '1150', '1200', '1210', '1220', '1230', '1240', '1250']
['1120', '1130', '1140', '1150', '1200', '1210', '1220', '1230', '1240', '1250']

--- Loading UV-Data.. ---

(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)
(1562, 561)

--- Sequence 0 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 0 ---

Dict: (438141, 66)
Test Success..

--- Sequence 1 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 15 ---

Dict: (438141, 66)
Test Success..

--- Sequence 2 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 88 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 74 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 8 ---

Dict: (438141, 66)
Test Success..



Dict: (438141, 66)
Test Success..

--- Sequence 2 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 8 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 1 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 59 ---

Dict: (438141, 66)
Test Success..

--- Sequence 5 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 5 ---

Dict: (438141, 66)
Test Success..

--- Sequence 0 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 89 ---

Dict: (438141, 66)
Test Success..

--- Sequence 1 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 19 ---

Dict: (438141, 66)
Test Success..

--- Sequence 2 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 72 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 88 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)


...

20190908_1320_U2.csv
20190908_1320_U3.csv
20190908_1320_U4.csv
20190908_1320_U5.csv
20190908_1330_U0.csv
20190908_1330_U1.csv
20190908_1330_U2.csv
20190908_1330_U3.csv
20190908_1330_U4.csv
20190908_1330_U5.csv
20190908_1340_U0.csv
20190908_1340_U1.csv
20190908_1340_U2.csv
20190908_1340_U3.csv
20190908_1340_U4.csv
20190908_1340_U5.csv
20190908_1350_U0.csv
20190908_1350_U1.csv
20190908_1350_U2.csv
20190908_1350_U3.csv
20190908_1350_U4.csv
20190908_1350_U5.csv
20190908_1400_U0.csv
20190908_1400_U1.csv
20190908_1400_U2.csv
20190908_1400_U3.csv
20190908_1400_U4.csv
20190908_1400_U5.csv
20190908_1410_U0.csv
20190908_1410_U1.csv
20190908_1410_U2.csv
20190908_1410_U3.csv
20190908_1410_U4.csv
20190908_1410_U5.csv
20190908_1420_U0.csv
20190908_1420_U1.csv
20190908_1420_U2.csv
20190908_1420_U3.csv
20190908_1420_U4.csv
20190908_1420_U5.csv
20190908_1430_U0.csv
20190908_1430_U1.csv
20190908_1430_U2.csv
20190908_1430_U3.csv
20190908_1430_U4.csv
20190908_1430_U5.csv
20190908_1440_U0.csv
20190908_1440

Dict: (4381410, 66)
Train Success..
--- Active terms 88 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 83 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 49 ---

Dict: (438141, 66)
Test Success..

--- Sequence 5 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 78 ---

Dict: (438141, 66)
Test Success..

--- Sequence 0 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 6 ---

Dict: (438141, 66)
Test Success..

--- Sequence 1 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 41 ---

Dict: (438141, 66)
Test Success..

--- Sequence 2 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 62 ---

Dict: (438141, 66)
Test Success..

--- Sequence 3 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 76 ---

Dict: (438141, 66)
Test Success..

--- Sequence 4 ---

Dict: (4381410, 66)
Train Success..
--- Active terms 98 ---

Dict: (43814

20190908_2100_U0.csv
20190908_2100_U1.csv
20190908_2100_U2.csv
20190908_2100_U3.csv
20190908_2100_U4.csv
20190908_2100_U5.csv
20190908_2110_U0.csv
20190908_2110_U1.csv
20190908_2110_U2.csv
20190908_2110_U3.csv
20190908_2110_U4.csv
20190908_2110_U5.csv
20190908_2120_U0.csv
20190908_2120_U1.csv
20190908_2120_U2.csv
20190908_2120_U3.csv
20190908_2120_U4.csv
20190908_2120_U5.csv
20190908_2130_U0.csv
20190908_2130_U1.csv
20190908_2130_U2.csv
20190908_2130_U3.csv
20190908_2130_U4.csv
20190908_2130_U5.csv
20190908_2140_U0.csv
20190908_2140_U1.csv
20190908_2140_U2.csv
20190908_2140_U3.csv
20190908_2140_U4.csv
20190908_2140_U5.csv
20190908_2150_U0.csv
20190908_2150_U1.csv
20190908_2150_U2.csv
20190908_2150_U3.csv
20190908_2150_U4.csv
20190908_2150_U5.csv
20190908_2200_U0.csv
20190908_2200_U1.csv
20190908_2200_U2.csv
20190908_2200_U3.csv
20190908_2200_U4.csv
20190908_2200_U5.csv
20190908_2210_U0.csv
20190908_2210_U1.csv
20190908_2210_U2.csv
20190908_2210_U3.csv
20190908_2210_U4.csv
20190908_2210

20190908_1730_U3.csv
20190908_1730_U4.csv
20190908_1730_U5.csv
20190908_1740_U0.csv
20190908_1740_U1.csv
20190908_1740_U2.csv
20190908_1740_U3.csv
20190908_1740_U4.csv
20190908_1740_U5.csv
20190908_1750_U0.csv
20190908_1750_U1.csv
20190908_1750_U2.csv
20190908_1750_U3.csv
20190908_1750_U4.csv
20190908_1750_U5.csv
20190908_1800_U0.csv
20190908_1800_U1.csv
20190908_1800_U2.csv
20190908_1800_U3.csv
20190908_1800_U4.csv
20190908_1800_U5.csv
20190908_1810_U0.csv
20190908_1810_U1.csv
20190908_1810_U2.csv
20190908_1810_U3.csv
20190908_1810_U4.csv
20190908_1810_U5.csv
20190908_1820_U0.csv
20190908_1820_U1.csv
20190908_1820_U2.csv
20190908_1820_U3.csv
20190908_1820_U4.csv
20190908_1820_U5.csv
20190908_1830_U0.csv
20190908_1830_U1.csv
20190908_1830_U2.csv
20190908_1830_U3.csv
20190908_1830_U4.csv
20190908_1830_U5.csv
20190908_1840_U0.csv
20190908_1840_U1.csv
20190908_1840_U2.csv
20190908_1840_U3.csv
20190908_1840_U4.csv
20190908_1840_U5.csv
20190908_1850_U0.csv
20190908_1850_U1.csv
20190908_1850

20190908_1350_U5.csv
20190908_1400_U0.csv
20190908_1400_U1.csv
20190908_1400_U2.csv
20190908_1400_U3.csv
20190908_1400_U4.csv
20190908_1400_U5.csv
20190908_1410_U0.csv
20190908_1410_U1.csv
20190908_1410_U2.csv
20190908_1410_U3.csv
20190908_1410_U4.csv
20190908_1410_U5.csv
20190908_1420_U0.csv
20190908_1420_U1.csv
20190908_1420_U2.csv
20190908_1420_U3.csv
20190908_1420_U4.csv
20190908_1420_U5.csv
20190908_1430_U0.csv
20190908_1430_U1.csv
20190908_1430_U2.csv
20190908_1430_U3.csv
20190908_1430_U4.csv
20190908_1430_U5.csv
20190908_1440_U0.csv
20190908_1440_U1.csv
20190908_1440_U2.csv
20190908_1440_U3.csv
20190908_1440_U4.csv
20190908_1440_U5.csv
20190908_1450_U0.csv
20190908_1450_U1.csv
20190908_1450_U2.csv
20190908_1450_U3.csv
20190908_1450_U4.csv
20190908_1450_U5.csv
20190908_1500_U0.csv
20190908_1500_U1.csv
20190908_1500_U2.csv
20190908_1500_U3.csv
20190908_1500_U4.csv
20190908_1500_U5.csv
20190908_1510_U0.csv
20190908_1510_U1.csv
20190908_1510_U2.csv
20190908_1510_U3.csv
20190908_1510

In [292]:
# print(mae)
# print(csi)
# print(far)
# print(pod)
# print(ets)
# print(hss)
# print(bss)
# print(mcc)
# print(prec)
# print(rec)

In [213]:
# saveimages = 1
# nowcasts = advectSINDyUV(saveimages)
# csi,mae,far,pod,ets,hss,bss, acc, prec, rec, fsc,mcc = evaluate(nowcasts)

/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/SINDy/24HrOutputImages
Reading u,v from  /home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/SINDy/pred_images
20190908_0220_U0.csv
20190908_0230_U0.csv
20190908_0230_U1.csv
20190908_0240_U0.csv
20190908_0240_U1.csv
20190908_0240_U2.csv
20190908_0250_U0.csv
20190908_0250_U1.csv
20190908_0250_U2.csv
20190908_0250_U3.csv
20190908_0300_U0.csv
20190908_0300_U1.csv
20190908_0300_U2.csv
20190908_0300_U3.csv
20190908_0300_U4.csv
20190908_0310_U1.csv
20190908_0310_U2.csv
20190908_0310_U3.csv
20190908_0310_U4.csv
20190908_0310_U5.csv
20190908_0320_U2.csv
20190908_0320_U3.csv
20190908_0320_U4.csv
20190908_0320_U5.csv
20190908_0330_U3.csv
20190908_0330_U4.csv
20190908_0330_U5.csv
20190908_0340_U4.csv
20190908_0340_U5.csv
20190908_0350_U5.csv
20190908_1100_U0.csv
20190908_1110_U1.csv
20190908_1120_U2.csv
20190908_1130_U3.csv
20190908_1140_U4.csv
20190908_1150_U5.csv
Read u,v frames from SINDy


In [ ]:
# csi,mae,far,pod,ets,hss,bss, acc, prec, rec, fsc,mcc = evaluate(nowcasts)

In [ ]:
# print(acc)
# print(csi)
# # print(mae)

In [368]:

print(len(maelist))
print("\n\n")
print(csilist)
print("\n\n")
print(podlist)
print("\n\n")
print(farlist)

print(precisionlist)
print(recalllist)


67



[[0.7300773190906812, 0.6600289446205141, 0.6411564931447773, 0.622549799388444, 0.6028129347494147, 0.590141443177735], [0.717011751538892, 0.6670633581413524, 0.63415802621223, 0.6187187464368902, 0.6003030115639107, 0.6001093526030353], [0.7291450926245542, 0.6768316248596721, 0.6418131456544083, 0.6281415426748826, 0.6206434316353887, 0.6171917205612237], [0.7384739875266441, 0.6898107414964713, 0.6613939959781672, 0.6539040728367923, 0.6302909060598483, 0.6046293450657523], [0.7481428182489985, 0.7082604613486994, 0.68436232694728, 0.6664760398970615, 0.638749311116154, 0.6020612521567661], [0.7517117184715526, 0.7139907892647293, 0.6762007012128634, 0.6548962386511025, 0.6181448570064528, 0.5887619635690028], [0.755013482383614, 0.7058141098930661, 0.6700800565861814, 0.6287679761886683, 0.60185158355307, 0.5973544824477377], [0.7617993340884908, 0.7073067809264129, 0.6545913940851459, 0.6259024030323636, 0.6143713244024753, 0.6021092870989169], [0.7443915590467315, 0.67594

In [ ]:
# Just for debugging

# fileName = "20190908_030000.000"
# pred_time ="_0300"
# for i in range(hrlimit):
#     if pred_time==times[i]:
#         recent_time=times[i-1]
# print(recent_time)

In [369]:
import xlwt 
from xlwt import Workbook 
  
# Workbook is created 
wb = Workbook() 

 

In [370]:
def writeinexcelsheet(sheetname, wb, results):
     
    sheet1 = wb.add_sheet(sheetname) 
    sheet1.write(0, 0, 'Pred.no.') 
    sheet1.write(0, 1, 't (pred start time)') 
    sheet1.write(0, 2, 't + 10') 
    sheet1.write(0, 3, 't + 20') 
    sheet1.write(0, 4, 't + 30') 
    sheet1.write(0, 5, 't + 40') 
    sheet1.write(0, 6, 't + 50') 
    
    
    col = 0

    rows = len(results)
    cols = len(results[0])
    print(cols)
    for rowno in range(rows):
        
        sheet1.write(rowno+1,0,rowno+1)   
        for col in range(cols):
#             print(rowno+1,col+1,results[rowno][col])
            sheet1.write(rowno+1,col+1,results[rowno][col].astype('float64'))
               
#          sheet1.write(row, col, str(data))
#          print(row,col,data)
    
    
    

In [371]:

writeinexcelsheet('CSI',wb,csilist)
writeinexcelsheet('MAE',wb,maelist)
writeinexcelsheet('FAR',wb,farlist)
writeinexcelsheet('POD',wb,podlist)


writeinexcelsheet('ETS',wb,etslist)
writeinexcelsheet('HSS',wb,hsslist)
writeinexcelsheet('BSS',wb,bsslist)
writeinexcelsheet('ACC',wb,acclist)

writeinexcelsheet('Precision',wb,precisionlist)
writeinexcelsheet('Recall',wb,recalllist)
writeinexcelsheet('F-Score',wb,fsclist)
writeinexcelsheet('MCC',wb,mcclist)
# excelpath = excelpath + 'resultsSINDy_0220_index34.xls'
excelpath = excelpath + 'resultsSINDy_1140_67items.xls'
wb.save(excelpath)


# excelpath = "/home/divya/divya/OneFullDayData_7TestCases_WNIMar5/24hroutputs/20190908/SINDy/eval_results/"
# excelpath = excelpath + 'resultsSINDy_13Apr2021.xls' # write first 34 predictions
# wb.save(excelpath)
    
# writeinexcelsheet('FAR',wb,farSINDyInterCVADV)
# writeinexcelsheet('POD',wb,podSINDyInterCVADV)
# writeinexcelsheet('HSS',wb,hssSINDyInterCVADV)

6
6
6
6
6
6
6
6
6
6
6
6


In [336]:
print(predtimelist)


['0540', '0550', '0600', '0610', '0620', '0630', '0640', '0650', '0700', '0710', '0720', '0730', '0740', '0750', '0800', '0810', '0820', '0830', '0840', '0850']


In [ ]:
#just for checking
# array = [[1,1,1,1,1,1],
#          [2,2,2,2,2,2],
#          [3,3,3,3,3,3],
#          [4,4,4,4,4,4]]


# writeinexcelsheet('nkkkkrrr',wb,array)


In [ ]:
#First 20 items
#['0220', '0230', '0240', '0250', '0300', '0310', '0320', '0330', '0340', '0350', '0400', '0410', '0420', '0430', '0440', '0450', '0500', '0510', '0520', '0530']
# index 14 to 34
#index 34 to 54
# ['0540', '0550', '0600', '0610', '0620', '0630', '0640', '0650', '0700', '0710', '0720', '0730', '0740', '0750', '0800', '0810', '0820', '0830', '0840', '0850']

In [330]:
oldpred_time = pred_time
oldindex =index
print("\old values")
print("old pred_time = ",pred_time)
print("old index = ",index)
index = 34
pred_time = times[index]
print("\n new pred time = ",pred_time)
templimit= 54

\old values
old pred_time =  0540
old index =  34

 new pred time =  0540


In [362]:
print(index)
print(pred_time)

70
1140
